In [1]:
import os
# Создаем spark сессию
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_func

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [11]:
DATA_DIR = 'D:/Datasets/ml-latest/'

In [12]:
# Сначала посмотрим на общее распределение тегов
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    .sample(withReplacement=False, fraction=1.0, seed=0)
    .withColumn('rating_datetime', sql_func.from_unixtime('timestamp'))
    .drop('timestamp')
    .cache()
)

In [16]:
# Файл с фильмами не большой, так, что его можно считать полностью
# двже, если памяти доступно немного
movie_genres = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'movies.csv'),
        header=True,
        inferSchema=True
    )
    # парсим информацию о жанрах,
    .withColumn('genres_array', sql_func.split('genres', '\|'))
    .select('movieId', sql_func.explode('genres_array').alias('genre'))
    .cache()
)

In [18]:
# у нас есть фильмы без меток
print('фильмов с жанрами', movie_genres.select('movieId').distinct().count())
print('фильмов с оценками', ratings.select('movieId').distinct().count())

фильмов с жанрами 58098
фильмов с оценками 53889


In [22]:
# создадим 'профиль пользваотеля' (жанровые предпоятения)
# набор средних ценок одного жанра
user_profiles = (
    ratings
    .join(movie_genres, 'movieId')
    .groupBy('userId', 'genre')
    .agg(sql_func.avg('rating').alias('genre_rating'))
    .cache()
)


In [24]:
# посмотрим, как может выглядеть профиль одного из пользователей
(
    user_profiles
    .where('userId == 23')
    .orderBy(sql_func.desc('genre_rating'))
    .show()
)

+------+---------+------------------+
|userId|    genre|      genre_rating|
+------+---------+------------------+
|    23|Adventure|               5.0|
|    23|  Romance| 4.166666666666667|
|    23|      War|               4.0|
|    23|   Horror|               4.0|
|    23|  Musical|               4.0|
|    23| Thriller|               4.0|
|    23|   Comedy|               3.8|
|    23|   Action|3.6666666666666665|
|    23|    Drama|            3.5625|
|    23|    Crime|              3.25|
|    23|   Sci-Fi|               3.0|
+------+---------+------------------+



In [27]:
import numpy as np

# предсказываем оценку фильма как среднее по средним оценкам жанров данного пользователя
predictions = (
    ratings
    .join(movie_genres, 'movieId', how='left')
    .join(user_profile, ['userId', 'genre'], how='left')
    .groupBy('userId', 'movieId', 'rating')
    .agg(sql_func.avg('genre_rating').alias('prediction'))
)
RMSE = np.sqrt(
    predictions
    .select(
        sql_func.pow(predictions.prediction - predictions.rating, 2)
        .alias('sqared_error')
    )
    .agg(sql_func.avg('sqared_error'))
    .first()[0]
)

In [28]:
# мы получили точность хуже, чем для линейной модели на средних весах
# но лучше, чем просто на модели со средними весами
print('точность', RMSE)

точность 0.8867015874123608
